In [38]:
!pip install brownie

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for brownie: filename=Brownie-0.5.1-py3-none-any.whl size=40694 sha256=a5bb7e069c931577de5218ef1ee42138ffec75f4c943851311274aac404ed703
  Stored in directory: /root/.cache/pip/wheels/6f/cb/31/7a689055b3614f8efae433145306ff0c6ebea2bc92b160c264
Successfully built brownie


## GET pool address & reserves

In [31]:
%load_ext dotenv
%dotenv .env -o -v

from web3 import Web3, HTTPProvider
import json, os, time

RPC_URL = os.getenv("GOERLI_RPC")
FACTORY_ADDRESS = "0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f" # UniswapV2Factory
ROUTER_ADDRESS = "0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D" # UniswapV2Router
ZETA_ADDRESS = "0xCc7bb2D219A0FC08033E130629C2B854b7bA9195" # ZETA on goerli
WETH_ADDRESS ="0xB4FBF271143F4FBf7B91A5ded31805e42b2208d6"# WETH on goerli

with open("abis/IUniswapV2Pair.json", "r") as f:
    IUniswapV2Pair =json.loads(f.read())
with open("abis/IUniswapV2Factory.json", "r") as f:
    IUniswapV2Factory =json.loads(f.read())
with open("abis/IUniswapV2Router02.json", "r") as f:
    IUniswapV2Router02 =json.loads(f.read())
with open("abis/IERC20.json", "r") as f:
    IERC20 =json.loads(f.read())

w3 = Web3(HTTPProvider(RPC_URL))
factory = w3.eth.contract(address = FACTORY_ADDRESS , abi = IUniswapV2Factory["abi"])
router = w3.eth.contract(address = ROUTER_ADDRESS , abi = IUniswapV2Router02["abi"])

eth_dai_address = factory.functions.getPair(ZETA_ADDRESS , WETH_ADDRESS).call() # eth_daiのコントラクトアドレスを取得
eth_dai = w3.eth.contract(address = eth_dai_address, abi = IUniswapV2Pair["abi"]) # eth_daiのコントラクトインスタンスを取得
print("")
print("token0 = ",  eth_dai.functions.token0().call())
print("token1 = ",  eth_dai.functions.token1().call())

reserves = eth_dai.functions.getReserves().call()
print("")
print("token0 reserver = ", reserves[0]*10**(-18))
print("token1 reserver = ", reserves[1]*10**(-18))
print("block.timestamp = ", reserves[2])
print("token0 price = ", reserves[1]/reserves[0]) # DAI/ETH price in this case

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv

token0 =  0xB4FBF271143F4FBf7B91A5ded31805e42b2208d6
token1 =  0xCc7bb2D219A0FC08033E130629C2B854b7bA9195

token0 reserver =  6262.467355434803
token1 reserver =  938548.3020116354
 block.timestamp =  1682048280
token0 price =  149.86877355889578


oken0 reserver =  6263.410192658549
token1 reserver =  938406.2974996647
 block.timestamp =  1682048028
token0 price =  149.82354159074345

In [44]:
weth = w3.eth.contract(address = WETH_ADDRESS, abi = IERC20["abi"])
dir(weth.functions.approve(ROUTER_ADDRESS, weth_amount))

['__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_transaction',
 '_encode_transaction_data',
 '_estimate_gas',
 '_get_call_txparams',
 '_return_data_normalizers',
 '_set_function_info',
 '_transact',
 'abi',
 'address',
 'args',
 'arguments',
 'build_transaction',
 'call',
 'contract_abi',
 'decode_tuples',
 'estimate_gas',
 'factory',
 'fn_name',
 'function_identifier',
 'get_fallback_function',
 'get_receive_function',
 'kwargs',
 'selector',
 'transact',
 'transaction',
 'w3']

## approve

In [6]:
my_address = os.getenv("ADDRESS")
my_private_key = os.getenv("PRIVATE_KEY")
weth = w3.eth.contract(address = WETH_ADDRESS, abi = IERC20["abi"])

weth_amount = w3.to_wei(0.1, "ether") # スワップしたい量

nonce = w3.eth.get_transaction_count(my_address)
tx_approve = weth.functions.approve(ROUTER_ADDRESS, weth_amount).build_transaction(\
    {'chainId': 5, \
     'type': 2, \
     'gas': 10**6, \
    'maxFeePerGas': w3.to_wei(100, 'gwei'),\
    'nonce': nonce, })
signed_tx_approve = w3.eth.account.sign_transaction(tx_approve, private_key = my_private_key)
w3.eth.send_raw_transaction(signed_tx_approve.rawTransaction)

HexBytes('0x373fcb56abeef9e30988ee9cf3585eec31b1f0b54fdea1d844fd2e7789e5ef61')

## swapExactTokensForTokens (WETH to ZETA)

In [9]:
amountIn = weth_amount
amountOutMin =int(reserves[0] / reserves[1] * amountIn * 0.99) # 1%以内のスリッページで実行
path = [WETH_ADDRESS, ZETA_ADDRESS]
to = my_address
deadline = int(time.time() + 60*2) # 2分以内に実行されなければrevert
nonce = w3.eth.get_transaction_count(my_address)
tx_swap = router.functions.swapExactTokensForTokens(amountIn, amountOutMin, path, to, deadline).build_transaction(\
    {'chainId': 5, \
     'type': 2, \
     'gas': 10**6, \
    'maxFeePerGas': w3.to_wei('100', 'gwei'),\
    'nonce': nonce, })
signed_tx_swap = w3.eth.account.sign_transaction(tx_swap, private_key = my_private_key)
w3.eth.send_raw_transaction(signed_tx_swap.rawTransaction)

HexBytes('0x4a3b9c2b5b5437900c514fc675298281fe42026d278fee5e10240c3164afac82')

## swapETHForExactTokens

In [36]:
amountIn = w3.to_wei(0.01, "ether")
path = [WETH_ADDRESS, ZETA_ADDRESS]
amountOutMin = int(reserves[1] / reserves[0] * amountIn * 0.95) # 5%以内のスリッページで実行
to = my_address
deadline = int(time.time() + 60 * 2) # 2分以内に実行されなければrevert
nonce = w3.eth.get_transaction_count(my_address)
tx_swap = router.functions.swapETHForExactTokens(amountOutMin, path, to, deadline).build_transaction(\
    {'chainId': 5, \
     'type': 2, \
     'gas': 10**6, \
    'maxFeePerGas': w3.to_wei('100', 'gwei'),\
    'nonce': nonce, 
    'value': amountIn
    })
signed_tx_swap = w3.eth.account.sign_transaction(tx_swap, private_key = my_private_key)
w3.eth.send_raw_transaction(signed_tx_swap.rawTransaction)

HexBytes('0xf76768d5018d037ab11ed0b00560e1fbe15764d3616543c748c7f96f74cee670')

## swapExactETHForTokens

In [42]:
amountOutMin = 0
path = [WETH_ADDRESS, ZETA_ADDRESS]
to = my_address
deadline = int(time.time() + 60 * 2) # 2分以内に実行されなければrevert
nonce = w3.eth.get_transaction_count(my_address)
tx_swap = router.functions.swapExactETHForTokens(amountOutMin, path, to, deadline).build_transaction(\
    {'chainId': 5, \
     'type': 2, \
     'gas': 10**6, \
    'maxFeePerGas': w3.to_wei('100', 'gwei'),\
    'nonce': nonce, 
    'value': w3.to_wei(0.01, "ether")
    })
signed_tx_swap = w3.eth.account.sign_transaction(tx_swap, private_key = my_private_key)
w3.eth.send_raw_transaction(signed_tx_swap.rawTransaction)

HexBytes('0x8957efccf10fd621fd866bb138a03e4b3299537d727d8726aacb7fee819e562d')